# Math Question Answer Verification Competition

## Starter Code

Borrowed from [official Unsloth implementation](https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing#scrollTo=MKX_XKs_BNZR)

In [1]:
# %%capture
# This cell will take time
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Found existing installation: unsloth 2024.11.7
Uninstalling unsloth-2024.11.7:
  Successfully uninstalled unsloth-2024.11.7
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-6p3ky96t/unsloth_8f796a1cc8b24bf38e01a8f8003fbf78
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-6p3ky96t/unsloth_8f796a1cc8b24bf38e01a8f8003fbf78
  Resolved https://github.com/unslothai/unsloth.git to commit f26d4e739ed507de7a9088da53d10fd02f58d160
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.11.7-py3-none-any.whl size=163138 sha256=d63991c73c4d23e85a65f57552688df841815671f2016431008b048518d89c10
  Stored in directory: /tmp/pip-ephem-wheel-cache-5jbzruud/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth


In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.0. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## Load model and wrap with LoRA adapters

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Competition dataset

In [5]:
# download and load competition dataset

from datasets import load_dataset
dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp")
# print and see dataset
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'is_correct', 'answer', 'solution'],
        num_rows: 1000000
    })
    test: Dataset({
        features: ['question', 'is_correct', 'answer', 'solution'],
        num_rows: 10000
    })
})

In [6]:
prompt = """You are a highly skilled mathematician tasked with verifying the correctness of mathematical solutions. Your task is to determine if the given answer to a mathematics question is correct or incorrect. Yout Instructions: 1. Carefully analyze the question and the provided answer 2. Verify the mathematical steps and logic 3. Check for any calculation errors 4. Respond with ONLY 'True' if the answer is completely correct, or 'False' if there are any errors. Below is Question and Answer.

### Question:
{}

### Answer:
{}

### Explainaition
{}

### Output:
{}"""





EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    question = examples["question"]
    ans       = examples["answer"]
    solu       = examples["solution"]
    output      = examples["is_correct"]
    texts = []
    for instruction, input,input2, output in zip(question, ans, solu, output):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, input, input2, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }




In [7]:
# Process the training dataset and generate prompt for each datapoint

train_dataset = dataset['train'].map(formatting_prompts_func, batched = True,)

In [8]:
#print a smaple training example
train_dataset['text'][0]

"You are a highly skilled mathematician tasked with verifying the correctness of mathematical solutions. Your task is to determine if the given answer to a mathematics question is correct or incorrect. Yout Instructions: 1. Carefully analyze the question and the provided answer 2. Verify the mathematical steps and logic 3. Check for any calculation errors 4. Respond with ONLY 'True' if the answer is completely correct, or 'False' if there are any errors. Below is Question and Answer.\n\n### Question:\nWhat is the radius of the circle inscribed in triangle $ABC$ if $AB = 22, AC=12,$ and $BC=14$? Express your answer in simplest radical form.\n\n### Answer:\n3.16227766016838\n\n### Explainaition\n\n### Output:\nTrue<|end_of_text|>"

## SFT

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

training_args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8,
        warmup_steps = 30,
        #num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 3000,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01, # 0.05
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none" # Use this for WandB etc
)



trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_args
)



max_steps is given, it will override any value given in num_train_epochs


In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 8
\        /    Total batch size = 32 | Total steps = 3,000
 "-____-"     Number of trainable parameters = 83,886,080


Step,Training Loss
1,2.108500
2,2.146500
3,2.149400
4,2.070800
5,1.989600
6,1.814600
7,1.734600
8,1.394900
9,1.240400
10,0.969300


## inference

In [11]:
# Sample inferene data point

test_dataset = dataset['test']


sample_ques = test_dataset['question'][0]
sample_ans = test_dataset['answer'][0]
sample_solu = test_dataset['solution'][0]



In [12]:
# Running inference on single test
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
input_prompt = prompt.format(
        sample_ques, # ques
        sample_ans, # given answer
        sample_solu, # solution
        "", # output - leave this blank for generation! LLM willl generate is it is True or False
    )

print("Input Prompt:\n", input_prompt)
inputs = tokenizer(
    [input_prompt],
    return_tensors = "pt"
).to("cuda")



input_shape = inputs['input_ids'].shape
input_token_len = input_shape[1] # 1 because of batch
# outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
# you can get the whole generated text by uncommenting the below line
# text_generated = tokenizer.batch_decode([outputs, skip_special_tokens=True)




outputs = model.generate(
    **inputs,
    max_new_tokens = 64,
    use_cache = True,
    temperature = 0.1,     # low temperature
    top_p = 0.9,
    do_sample = True,      # use sampling
    num_beams = 1,        # set to 1 to disable beam search
    repetition_penalty = 1.2  # add repetition penalty
)

response = tokenizer.batch_decode([outputs[0][input_token_len:]], skip_special_tokens=True)
response


Input Prompt:
 You are a highly skilled mathematician tasked with verifying the correctness of mathematical solutions. Your task is to determine if the given answer to a mathematics question is correct or incorrect. Yout Instructions: 1. Carefully analyze the question and the provided answer 2. Verify the mathematical steps and logic 3. Check for any calculation errors 4. Respond with ONLY 'True' if the answer is completely correct, or 'False' if there are any errors. Below is Question and Answer.

### Question:
The Parker family needs to leave the house by 5 pm for a dinner party. Mrs. Parker was waiting to get into the bathroom at 2:30 pm. Her oldest daughter used the bathroom for 45 minutes and her youngest daughter used the bathroom for another 30 minutes. Then her husband used it for 20 minutes. How much time will Mrs. Parker have to use the bathroom to leave on time?

### Answer:
205

### Explainaition

### Output:



['False']

In [17]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=64,
    temperature=0.1,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

def generate_prompts(test_dataset):
    for i in range(len(test_dataset)):
        yield prompt.format(
            test_dataset['question'][i],
            test_dataset['answer'][i],
            test_dataset['solution'][i],
            ""
        )

# restult
results = []
test_dataset = dataset['test']

test_dataset_test = test_dataset.select(range(1000))

# show progress
for i, out in enumerate(tqdm(pipe(
    generate_prompts(test_dataset),
    batch_size=8  
), total=len(test_dataset))):

    
    generated_text = out[0]['generated_text']

   
    prediction = generated_text.split('Output:')[1].strip().lower()

    
    if 'true' in prediction:
        is_correct = 'True'
    elif 'false' in prediction:
        is_correct = 'False'
    else:
        is_correct = 'False'  

    # add result
    results.append({
        'ID': i,
        'is_correct': is_correct
    })

# save result
df = pd.DataFrame(results)
df.to_csv('submission.csv', index=False)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausa

## saving model

In [15]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [16]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference


==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.0. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
